The convex hull of the set is the smallest convex polygon that enclosed all the points in a plane. The vertice maximize the area while minimize the circumferences. There are two main types of convex hull algorithms:

* Graham Scan
* Jarvis March

**references:**
1. https://www.youtube.com/watch?v=9rQMLpQn5xQ&t=2s
2. http://www.math.ucsd.edu/~ronspubs/72_10_convex_hull.pdf
3. https://demonstrations.wolfram.com/GrahamScanToFindTheConvexHullOfASetOfPointsIn2D/

# **Graham Scan**

The algorithm steps as following:
1. Find the point which has lowest Y coordinate. If two points with the same y value, then the point with smaller x coordinate value is considered. 
2. Calculate the angle between the most lowest point and other points, then sort the points by angle in counterclockwise order.
3. Append the points which is in counterclockwise from each point to each point. Remove the previous points from the stack if it make clockwise.

Another method is based on signed of parallelogram. If the area is positive, then it is counterclockwise turn, whereas if the area is negative, it is clockwise turn.

In [1]:
# This code is contributed by Kevin Joshi
from functools import cmp_to_key

# A class used to store the x and y coordinates of points
class Point:
	def __init__(self, x = None, y = None):
		self.x = x
		self.y = y

# A global point needed for sorting points with reference
# to the first point
p0 = Point(0, 0)

# A utility function to find next to top in a stack
def nextToTop(S):
	return S[-2]

# A utility function to return square of distance
# between p1 and p2
def distSq(p1, p2):
	return ((p1.x - p2.x) * (p1.x - p2.x) + (p1.y - p2.y) * (p1.y - p2.y))

# To find orientation of ordered triplet (p, q, r).
# The function returns following values
# 0 --> p, q and r are collinear
# 1 --> Clockwise
# 2 --> Counterclockwise
def orientation(p, q, r):
	val = ((q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y))
	if val == 0:
		return 0 # collinear
	elif val > 0:
		return 1 # clock wise
	else:
		return 2 # counterclock wise

# A function used by cmp_to_key function to sort an array of
# points with respect to the first point
def compare(p1, p2):

	# Find orientation
	o = orientation(p0, p1, p2)
	if o == 0:
		if distSq(p0, p2) >= distSq(p0, p1):
			return -1
		else:
			return 1
	else:
		if o == 2:
			return -1
		else:
			return 1

# Prints convex hull of a set of n points.
def convexHull(points, n):

	# Find the bottommost point
	ymin = points[0].y
	min = 0
	for i in range(1, n):
		y = points[i].y

		# Pick the bottom-most or chose the left
		# most point in case of tie
		if ((y < ymin) or
			(ymin == y and points[i].x < points[min].x)):
			ymin = points[i].y
			min = i

	# Place the bottom-most point at first position
	points[0], points[min] = points[min], points[0]

	# Sort n-1 points with respect to the first point.
	# A point p1 comes before p2 in sorted output if p2
	# has larger polar angle (in counterclockwise
	# direction) than p1
	p0 = points[0]
	points = sorted(points, key=cmp_to_key(compare))

	# If two or more points make same angle with p0,
	# Remove all but the one that is farthest from p0
	# Remember that, in above sorting, our criteria was
	# to keep the farthest point at the end when more than
	# one points have same angle.
	m = 1 # Initialize size of modified array
	for i in range(1, n):
	
		# Keep removing i while angle of i and i+1 is same
		# with respect to p0
		while ((i < n - 1) and
		(orientation(p0, points[i], points[i + 1]) == 0)):
			i += 1

		points[m] = points[i]
		m += 1 # Update size of modified array

	# If modified array of points has less than 3 points,
	# convex hull is not possible
	if m < 3:
		return

	# Create an empty stack and push first three points
	# to it.
	S = []
	S.append(points[0])
	S.append(points[1])
	S.append(points[2])

	# Process remaining n-3 points
	for i in range(3, m):
	
		# Keep removing top while the angle formed by
		# points next-to-top, top, and points[i] makes
		# a non-left turn
		while ((len(S) > 1) and
		(orientation(nextToTop(S), S[-1], points[i]) != 2)):
			S.pop()
		S.append(points[i])

	# Now stack has the output points,
	# print contents of stack
	while S:
		p = S[-1]
		print("(" + str(p.x) + ", " + str(p.y) + ")")
		S.pop()

# Driver Code
input_points = [(0, 3), (1, 1), (2, 2), (4, 4),
				(0, 0), (1, 2), (3, 1), (3, 3)]
points = []
for point in input_points:
	points.append(Point(point[0], point[1]))
n = len(points)
convexHull(points, n)

(0, 3)
(4, 4)
(3, 1)
(0, 0)


# **Jarvis March**

The algorithm steps are following:

1. Find the point which has lowest Y coordinate as the first vertex.
2. Select the point which has smallest counterclockwise angle in references to the previous vertex.
3. Repeat the second step until it reach the starting vertex.

In [3]:
# This code is contributed by Akarsh Somani, IIIT Kalyani

# point class with x, y as point
class Point:
	def __init__(self, x, y):
		self.x = x
		self.y = y

def Left_index(points):
	
	'''
	Finding the left most point
	'''
	minn = 0
	for i in range(1,len(points)):
		if points[i].x < points[minn].x:
			minn = i
		elif points[i].x == points[minn].x:
			if points[i].y > points[minn].y:
				minn = i
	return minn

def orientation(p, q, r):
	'''
	To find orientation of ordered triplet (p, q, r).
	The function returns following values
	0 --> p, q and r are collinear
	1 --> Clockwise
	2 --> Counterclockwise
	'''
	val = (q.y - p.y) * (r.x - q.x) - \
		(q.x - p.x) * (r.y - q.y)

	if val == 0:
		return 0
	elif val > 0:
		return 1
	else:
		return 2

def convexHull(points, n):
	
	# There must be at least 3 points
	if n < 3:
		return

	# Find the leftmost point
	l = Left_index(points)

	hull = []
	
	'''
	Start from leftmost point, keep moving counterclockwise
	until reach the start point again. This loop runs O(h)
	times where h is number of points in result or output.
	'''
	p = l
	q = 0
	while(True):
		
		# Add current point to result
		hull.append(p)

		'''
		Search for a point 'q' such that orientation(p, q,
		x) is counterclockwise for all points 'x'. The idea
		is to keep track of last visited most counterclock-
		wise point in q. If any point 'i' is more counterclock-
		wise than q, then update q.
		'''
		q = (p + 1) % n

		for i in range(n):
			
			# If i is more counterclockwise
			# than current q, then update q
			if(orientation(points[p],
						points[i], points[q]) == 2):
				q = i

		'''
		Now q is the most counterclockwise with respect to p
		Set p as q for next iteration, so that q is added to
		result 'hull'
		'''
		p = q

		# While we don't come to first point
		if(p == l):
			break

	# Print Result
	for each in hull:
		print(points[each].x, points[each].y)

# Driver Code
input_points = [(0, 3), (1, 1), (2, 2), (4, 4),
				(0, 0), (1, 2), (3, 1), (3, 3)]
points = []
for point in input_points:
	points.append(Point(point[0], point[1]))

convexHull(points, len(points))

0 3
0 0
3 1
4 4
